In [1]:
from typing import Tuple

import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'germany'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

model = Prophet().fit(df_train)

02:59:30 - cmdstanpy - INFO - Chain [1] start processing
02:59:31 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,3.077382,2.760480,3.502377,3.077382,3.077382,0.063457,0.063457,0.063457,0.063457,0.063457,0.063457,0.0,0.0,0.0,3.140839
1,2022-04-01,3.065524,2.752353,3.490742,3.065524,3.065524,0.050725,0.050725,0.050725,0.050725,0.050725,0.050725,0.0,0.0,0.0,3.116249
2,2022-05-01,3.054048,2.662282,3.457991,3.052158,3.055121,0.019789,0.019789,0.019789,0.019789,0.019789,0.019789,0.0,0.0,0.0,3.073837
3,2022-06-01,3.042190,2.674784,3.479619,3.033990,3.047667,0.020835,0.020835,0.020835,0.020835,0.020835,0.020835,0.0,0.0,0.0,3.063025
4,2022-07-01,3.030714,2.676097,3.425945,3.013907,3.045585,0.020837,0.020837,0.020837,0.020837,0.020837,0.020837,0.0,0.0,0.0,3.051551
5,2022-08-01,3.018856,2.661225,3.406318,2.988087,3.045242,0.027228,0.027228,0.027228,0.027228,0.027228,0.027228,0.0,0.0,0.0,3.046084
6,2022-09-01,3.006998,2.679762,3.439579,2.965284,3.045283,0.037431,0.037431,0.037431,0.037431,0.037431,0.037431,0.0,0.0,0.0,3.044429
7,2022-10-01,2.995522,2.671510,3.447156,2.937716,3.051705,0.041608,0.041608,0.041608,0.041608,0.041608,0.041608,0.0,0.0,0.0,3.037130
8,2022-11-01,2.983664,2.604414,3.433996,2.905079,3.059079,0.031440,0.031440,0.031440,0.031440,0.031440,0.031440,0.0,0.0,0.0,3.015104
9,2022-12-01,2.972188,2.595402,3.423049,2.873124,3.068694,0.028207,0.028207,0.028207,0.028207,0.028207,0.028207,0.0,0.0,0.0,3.000395


In [11]:
rmse

0.07215052847565874

In [12]:
with open(f'{country}_prophet_base_model.json', 'w') as f:
    f.write(model_to_json(model))

In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [14]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,2.937761,2.659952,3.462228,2.937761,2.937761,0.116649,0.116649,0.116649,0.116649,0.116649,0.116649,0.0,0.0,0.0,3.054410
1,2023-04-01,2.925903,2.584905,3.386250,2.925903,2.925903,0.071158,0.071158,0.071158,0.071158,0.071158,0.071158,0.0,0.0,0.0,2.997061
2,2023-05-01,2.914427,2.542403,3.324068,2.914427,2.914427,0.000531,0.000531,0.000531,0.000531,0.000531,0.000531,0.0,0.0,0.0,2.914958
